This script runs a 1D simulation of interacting colonies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
Lx = 15.
nx = 500

x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
domain = de.Domain([x_basis], grid_dtype=np.float64)

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
Km =  params['K'].flat[0]
g = params['g'].flat[0]


problem = de.IVP(domain, variables=['rho1','rho2','rho12'])
problem.parameters['D0'] = Diff
problem.parameters['g0'] = g
problem.parameters['K'] = Km

problem.substitutions['rho_b'] = "rho1+rho2  + rho12" 
problem.substitutions["Lap(A)"] = "dx(dx(A))"

problem.substitutions["grow_non_lin(A,B)"] = "-A*B"

problem.add_equation("dt(rho1) - g0*rho1 - D0*Lap(rho1) = g0*grow_non_lin(rho1,rho_b) - K*rho1*rho2")
problem.add_equation("dt(rho2) - g0*rho2 - D0*Lap(rho2) = g0*grow_non_lin(rho2,rho_b) - K*rho1*rho2")
problem.add_equation("dt(rho12)  = 2*K*rho1*rho2")

In [ ]:
ts = de.timesteppers.RK443
solver =  problem.build_solver(ts)

In [ ]:
x = domain.grid(0)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']
rho12 = solver.state['rho12']

def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob(x,y,x0,y0,r0,w):
    return 0.5*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

def r(x,x0):
    return np.sqrt((x-x0)**2)

def blob(x,x0,r0,w):
    return 0.5*(1 - np.tanh((r(x,x0)-r0)/w))

r0 = 0.75 #mm
w = 0.15 #mm
sep = 9.0 #mm
rho1['g'] = blob(x,Lx/2 - sep/2,r0,w)
rho2['g'] = blob(x,Lx/2 + sep/2,r0,w)

In [ ]:
solver.stop_sim_time = 200
dt = 0.1*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('1DInteract', sim_dt=1.0, max_writes=500)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rho12')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))


In [ ]:
from dedalus.tools import post
post.merge_process_files("1DInteract", cleanup=True)